In [1]:
import random
import numpy as np
from IPython.display import clear_output

In [ ]:
GRID = 4
GOAL = 2048

def success(board):
    return GOAL in board

def failure(board):
    return 0 not in board

def ended(board):
    return success(board) or failure(board)

def state(board):
    if success(board):
        return 'win'
    if failure(board):
        return 'lose'
    return 'play'

def coord():
    """a random tile coordinate
    """
    x = random.randint(0, GRID - 1)
    y = random.randint(0, GRID - 1)
    return x, y

def occupied(board, x, y):
    return board[x, y]

def spawn(board):
    """place 2 or 4 on a random free tile
    """
    if ended(board):
        return board
    board = np.copy(board)
    x, y = coord()
    while occupied(board, x, y):
        x, y = coord()
    board[x, y] = 2 if random.random() < 0.9 else 4
    return board

def orient(board, direction):
    """rotate/flip board to facilitate numpy slicing
    """
    board = np.copy(board)
    if direction == 'd':
        board = np.flipud(board)
    if direction in 'lr':
        board = np.transpose(board)
    if direction == 'r':
        board = np.fliplr(board)
        board = np.flipud(board)
    return board

def merge(board, func):
    """merge rows/columns when tilting
    """
    def _merge(from_, into):
        """move vector `from_` into vector `into`
        merging elements according to `func`"""
        zipped = map(lambda t: func(*t), zip(from_, into))
        return tuple(zip(*zipped))  # unzip

    board = np.copy(board)
    ops = list(range(GRID))
    # merge pairwise rows
    for i in range(GRID - 1):
        fr, to = ops[i], ops[i + 1] + 1
        board[fr:to] = np.array(_merge(*board[fr:to]))
    return board
    
def tilt(board, direction):
    """move/sum tiles based in a certain direction
    """
    def zeros(a, b):
        """shift left: (0, 2) -> (2, 0)"""
        return (b, 0) if a == 0 else (a, b)

    def equal(a, b):
        """add left: (2, 2) -> (4, 0)"""
        return (a + b, 0) if a == b else (a, b)

    board = orient(board, direction)
    for i in range(GRID - 1):
        board = merge(board, zeros)
    board = merge(board, equal)
    board = merge(board, zeros)
    return orient(board, direction)

def move(board, direction):
    """one game step
    """
    prev = board
    board = tilt(board, direction)
    # test for noop moves
    if np.array_equal(prev, board):
        return board, 'play'
    if ended(board):
        return board, state(board)
    return spawn(board), 'play'

def print_(board):
    n = len(str(GOAL)) + 1
    boundary = (n - 1) * '+-----' + '+'
    print(boundary)
    for row in board:
        nums = [f"{i: {n}d}" if i > 0 else n * ' ' for i in row]
        print(f"|{'|'.join(nums)}|")
        print(boundary)

def new():
    board = np.zeros((GRID, GRID), dtype=int)
    board = spawn(board)
    board = spawn(board)
    return board

def direction(board):
    """pick a direction.
    TODO: replace with AI ;~}
    """
    return input()

def play(board):
    state = 'play'
    while state == 'play':
        clear_output()
        print_(board)
        board, state = move(board, direction(board))
    print_(board)
    print(f'you {state}')


board = new()
play(board)

+-----+-----+-----+-----+
|    2|     |     |     |
+-----+-----+-----+-----+
|    8|    4|     |     |
+-----+-----+-----+-----+
|   64|    4|    2|     |
+-----+-----+-----+-----+
|  128|    8|    4|    2|
+-----+-----+-----+-----+
